In [1]:
# importing all the necessary libraries
import os 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader

from collections import OrderedDict #OrderedDict is a dictionary subclass that remembers the order that keys were first inserted.
import seaborn as sns # seaborn is a Python data visualization library based on matplotlib. Go to library for plotting.
import matplotlib.pyplot as plt
from timeit import default_timer as timer # to measure the time required to run a function.
from sklearn.metrics import roc_auc_score

# for showing the images
import cv2 # tool for image processing and performing computer vision tasks. Open-source library used to perform the task of image detection between NORMAL and PNEUMONIA
import random # to generate everytime you run the code a different set of images in each pertaining batch
random.seed(1) #used to generate random object in Python.

In [2]:
root = '../input/chest-xray-pneumonia/chest_xray/chest_xray/'
# (os.path.join() join one or more path components
train_dir = os.path.join(root, 'train')
val_dir = os.path.join(root, 'val')
test_dir = os.path.join(root, 'test')

In [3]:
classes = ['NORMAL', 'PNEUMONIA'] # defining the class name(s).

for c in classes: # created class
    target_path = os.path.join(train_dir, c) # path. join method combines one or more path names into a single path
    sample_normal = random.sample(os.listdir(os.path.join(train_dir, c)),6) # os.listdir used to get the list of all files and directories in the specified directory. os.path.join combines one or more path names into a single path
    f,ax = plt.subplots(2,3,figsize=(15,9)) # figure and axis' . A function that returns a tuple (an immutable collection of values seperated by comma and enclosed by parenthesis) containing a figure and axes object(s

    for i in range(6):
        im = cv2.imread(os.path.join(target_path, sample_normal[i])) # cv2. imread() method loads an image from the specified file
        ax[i//3,i%3].imshow(im) # Axes.imshow() function in axes module of matplotlib library is also used to display an image or data on a 2D regular roster
        ax[i//3,i%3].axis('off') #Rather than using plt.axis('off') you should use ax.axis('off') where ax is a matplotlib.axes object. To do this for your code you simple need to add axarr[3,3].axis('off') and so on for each of your subplots.
    f.suptitle("{} Lungs ".format(c), fontsize=25)
    plt.show()

In [4]:
data_transforms = {
    'train': transforms.Compose([ # stated below are some of the transforms to manipulate the data in the required format
        transforms.Resize((224, 224)),
        transforms.ToTensor(), # convert a PIL Image or numpy. ndarray to tensor 
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # Using the mean and std of Imagenet is a common practice
        # the images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
    ]),
    'test': transforms.Compose([ # stated below are some of the transforms to manipulate the data in the required format
        transforms.Resize((224,224)),
        transforms.ToTensor(), # convert a PIL Image or numpy. ndarray to tensor 
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # Using the mean and std of Imagenet is a common practice
        # the images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
    ]),
    'val': transforms.Compose([ # stated below are some of the transforms to manipulate the data in the required format
        transforms.Resize((224,224)),
        transforms.ToTensor(), # convert a PIL Image or numpy. ndarray to tensor 
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # Using the mean and std of Imagenet is a common practice
        # the images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
    ])
}

In [ ]:
batch_size = 32 # batch size defines the number of samples that will be propagated throughout the network
# batch size controls the accuracy of the estimate of the error gradient when training neural networks
# batch size of 32 means that 32 samples from the training dataset will be used to estimate the error gradient before the model weights are updated
data_sets = { # used datasets to train the model
    'train': torchvision.datasets.ImageFolder(train_dir,data_transforms['train']),
    'test': torchvision.datasets.ImageFolder(train_dir,data_transforms['test']),
    'val': torchvision.datasets.ImageFolder(train_dir,data_transforms['val']),
}

data_loaders = { # combines a dataset and a sampler, and provides an iterable over the given dataset
    # num_workers=2 we have 2 workers simultaneously putting data into RAM. We did not want to overload so we went with 2
    # By using shuffle=True you shuffle up the dataset which makes the training batches more generalized which in turn
    # makes the  'train' and 'test' models more generalized. We used 'shuffle=False' for 'val' as there
    # are 9 of each images in the NORMAL and PNEUMONIA folders
    'train': DataLoader(data_sets['train'], batch_size=batch_size, shuffle=True, num_workers=2),
    'val': DataLoader(data_sets['val'], batch_size=batch_size, shuffle=False, num_workers=2),
    'test': DataLoader(data_sets['test'], batch_size=batch_size, shuffle=True, num_workers=2)
}    

In [6]:
# created directories
dir_ = [train_dir, val_dir, test_dir]
x = []
y = []

for i in range(3): # created loop
    # listdir() method used to get the list of all files and directories in the specified directory.
    # If we don't specify any directory, then list of files and directories in the current working directory will be returned.
    target_normal = os.listdir(os.path.join(dir_[i], 'NORMAL'))
    target_pneumonia = os.listdir(os.path.join(dir_[i], 'PNEUMONIA'))
    # _.append() adds a single item to the existing list.
    # After executing the method append on the list the size of the list increases by one.
    x.append(len(target_normal))
    y.append(len(target_pneumonia))
    
    # 'os.path.basename' method returns a string value which represents the base name the specified path. 
    print("In {} folder... \n normal:{} \t pneumonia: {}".format(os.path.basename(dir_[i]), x[i], y[i]))

In [7]:
# We created a bar chart to depict the number of pictures in each cooresponding folder.
labels = ['train_file', 'val_file', 'test_file'] 
plt.figure(figsize=(12, 6))
width = 0.30
plt.bar(np.arange(len(x))- width/2, x, width, label="normal")
plt.bar(np.arange(len(x))+ width/2, y, width, label="pneumonia")
plt.xticks(np.arange(len(x)), labels, fontsize=14)

plt.title("Overall Depiction of Chest-xray-Pneumonia Dataset", fontsize=20) # title
plt.show() # Display of bar chart

In [8]:
model = models.densenet121(pretrained=True) # Input the densenet121 model

# To get the parameter count of each layer our PyTorch has model.parameters() that returns an
# iterator of both the parameter name and the parameter itself.
for param in model.parameters():
    param.requires_grad = True # All layers have the parameters modified during training as 
    # 'requires_grad' is set to true. 'Import torch', torchvision 'import torch.nn as nn' from collections import.

In [9]:
model.classifier = nn.Sequential(OrderedDict([ # Setting up to train our model(s).
    ('fcl1', nn.Linear(1024,256)), # nn. Linear(n,m) is a module that creates single layer feed forward network with n inputs and m output
    ('dp1', nn.Dropout(0.3)), # without nn.dropout run diverges a lot after just a few epochs. Hence it calls for the higher generalization error.
    ('r1', nn.ReLU()), # Relu is an activation function. After each layer, an activation function needs to be applied so as to make the network non-linear.
    ('fcl2', nn.Linear(256,32)), # nn. Linear(n,m) is a module that creates single layer feed forward network with n inputs and m output
    ('dp2', nn.Dropout(0.3)), # without nn.dropout run diverges a lot after just a few epochs. Hence it calls for the higher generalization error.
    ('r2', nn.ReLU()),  # Relu is an activation function. After each layer, an activation function needs to be applied so as to make the network non-linear.
    ('fcl3', nn.Linear(32,2)), # nn. Linear(n,m) is a module that creates single layer feed forward network with n inputs and m output
    ('out', nn.LogSoftmax(dim=1)), # a Softmax-based Activation Function is the logarithm of a Softmax Function
]))  # We used Log Softmax as it is advantageous over softmax for numerical stability, optimisation and heavy penalisation for highly incorrect class.

In [15]:
history = []

def train(model, train_loader, val_loader, criterion, optimizer, scheduler=None, train_on_gpu=False, num_epochs=30, save_file='model.pth'):
    
    overall_start = timer() # The time takes to run  
    valid_loss_min= np.Inf  # Initialize tracker for minimum validation loss
    
    #  'Epoch' - One training epoch means that the learning algorithm
    #            has made one pass through the training dataset.
    
    if train_on_gpu:
        model.cuda() # it is not DataLoaders job to send anything directly to GPU, we explicitly call cuda() for that 
    for epoch in (1, num_epochs): # loop 
        train_loss = 0
        val_loss = 0
        
        train_start = timer() 
              
        if scheduler is not None: # In event we used a scheduler otherwise we set it to None
            scheduler.step()
        
        # train region
        
        model.train()
        for data, target in train_loader:
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
                # cuda is used to set up and run CUDA operations.
                # It keeps track of the currently selected GPU, and
                # all CUDA tensors you allocate will by default be created on that device.
                # The selected device can be changed with a torch. cuda.
                
            optimizer.zero_grad() # Sets the gradients of all optimized torch

            # optimizer.zero_grad() clears x.grad for every parameter x in the optimizer.
            # It’s important to call this before loss.backward(), otherwise we accumulate
            # the gradients from multiple passes.
            output = model(data) # the output of the model data
            loss = criterion(output, target)
            loss.backward()  # computes the mean-squared error between the input and the target.
            optimizer.step() # After computing the gradients for all tensors in the model, calling 
                             # optimizer. step() makes the optimizer iterate over all parameters (tensors)
                             # it is supposed to update and use their internally stored grad to update
                             # their values.
            train_loss += loss.item() * data.size(0)
            #  loss. item() contains the loss of entire mini-batch, but divided by the batch size.
            #  That's why loss.item() is multiplied with batch size, given by inputs.
            
            # Track the training progress...
            print(
            f'Epoch: {epoch}\t {timer() - train_start:.2f} seconds elapsed in training epoch.',
            end='\r')
        
        
        print(f'Epoch: {epoch}\t {timer() - train_start:.2f} seconds elapsed in training epoch.')
        # val region 
                
        model.eval()
        number_correct, number_data = 0, 0
        for data, target in val_loader: # loop
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)
            
        # calculate accuracy
            # values , indices = torch.max(output , 1)  --> max in row
            _, pred = torch.max(output, 1)
            correct_tensor = pred.eq(target.data.view_as(pred)) # Computes element-wise equality
            correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu \
                                    else np.squeeze(correct_tensor.cpu().numpy()) # squeeze function in PyTorch is used for manipulating a tensor by dropping all its dimensions of inputs having size 1.
            number_correct += sum(correct) # Returns the sum of each row of the sparse tensor input in the given dimensions dim
            number_data += correct.shape[0]
            
            #Track eval & cal progress
            print(
            f'Epoch: {epoch}\t {timer() - eval_start:.2f} seconds elapsed in validation epoch.',
            end='\r')
        
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        accuracy = (100 * number_correct / number_data)
        print("Epoch: {} \n-----------------\n \tTraining Loss: {:.6f} \t Validation Loss: {:.6f} \t accuracy : {:.4f}% ".format(epoch, train_loss, val_loss, accuracy))
        if val_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, val_loss))
            torch.save(model.state_dict(), save_file)
            valid_loss_min = val_loss
            
        total_time = timer() - overall_start
        print(
            f'{total_time:.2f} total seconds elapsed. {(total_time) / (epoch):.2f} seconds per epoch.'
        )
        
        history.append({'train_loss': train_loss, 'val_loss': val_loss, 'acc' : accuracy})
        # to append or save the history data in the train, val, and acc files
               
    model.to('cpu') #loading to the CPU
    
    return torch.load(save_file) # load and save model file
        

In [14]:
train_on_gpu = torch.cuda.is_available() # Call function for running on GPU
if train_on_gpu: #Created a loop in the event we wish to use the GPU over the CPU, for faster results.
    print('GPU is  available. Training on GPU ...')
else:
    print('GPU is not available. Training on CPU ...')
    
# If you need to move a model to GPU via .cuda()
# please do so before constructing optimizers for it.
# Parameters of a model after .cuda() will be different
# objects with those before the call.    
    

In [ ]:
criterion = nn.NLLLoss() # 'nn. NLLLoss' expects the inputs to be log probabilities, while you are passing the probabilities into the criterion.
optimizer = optim.Adadelta(model.parameters()) # An adaptive learning rate method 'Adadelta Algorithm' as our optimizer
num_epochs = 2 # number of epochs we chose to test our neural network

# To construct an Optimizer you have to give it an iterable containing
# the parameters (all should be Variable s) to optimize. Then, you can
# specify optimizer-specific options such as the learning rate, etc.

model_state_dict = train(
                            model, #the model
                            # our two dataLoaders 'train' and 'val'
                            data_loaders['train'], 
                            data_loaders['val'],
                            criterion=criterion, # Creates a criterion that measures the mean absolute value between n elements in the input x and output y
                            optimizer=optimizer,# Optimization is the process of adjusting model parameters to reduce model error in each training step.
                            scheduler=None, # We avoided a scheduler out of good practice :-). Otherwise, they are used to adjust only the hyperparameter of learning rate in a model. Early stopping refers to another hyperparameter, the number of train epochs.
                            train_on_gpu=train_on_gpu,# Train on GPU
                            num_epochs=num_epochs, # An epoch is a measure of the number of times all training data is used once to update the parameters. We want our neural networks to train quickly.
                            )

model.load_state_dict(model_state_dict) # Saving and loading the model...

In [ ]:
# Defining test, printing
def test(model, test_loader, train_on_gpu, criterion, classes): 
    print('Commence Test')
    test_loss = 0
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))
    
    if train_on_gpu:
        model.cuda()
    
    # Evaluate model 
    test_start = timer()
    model.eval()
    cat_accuracy = {}
    for data, target in test_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)
        
        _, pred = torch.max(output, 1) 
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu \
                                else np.squeeze(correct_tensor.cpu().numpy())
        
        print(f'{timer() - test_start:.2f} seconds elapsed in test mode.',
            end='\r')
        
        for i in range(data.shape[0]):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    test_loss = test_loss / len(test_loader.dataset)
    print("Test Loss: {:.6f}".format(test_loss))
    print("Test Accuracy (Overall): %2d%% (%2d/%2d) \n ----------------------" % (100. * np.sum(class_correct) / np.sum(class_total),np.sum(class_correct), np.sum(class_total)))
    for i in range(len(classes)):
        if class_total[i] > 0:
            print('Test Accuracy of %s : %d%% (%2d/%2d)' % (classes[i], 100 * class_correct[i] / class_total[i],np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %s: N/A (no training examples)' % (classes[str(i+1)]))

In [ ]:
criterion = nn.NLLLoss() # Obtaining log-probabilities in a neural network is easily
                         # achieved by adding a LogSoftmax layer in the last layer of
                         # our network. 
test(model, data_loaders['test'], train_on_gpu, criterion, classes) # Testing and loading our model on the computers GPU.